In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from natsort import natsorted

In [2]:
data = pd.read_csv('acc_climbingup_forearm.csv')  

In [4]:
x = data['attr_x'].to_numpy()

In [7]:
x.mean()

-9.669502475190734